In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from statistics import NormalDist
from scipy.stats import norm
import scipy.special as sp

# Downloading the data:

In [2]:
X = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_3\initial_inputs.npy")
y = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\IMP-PCMLAI-capstone-initial_data\initial_data\function_3\initial_outputs.npy")

Here I'm appending the inputs resulted from the acquisition function below, so everytime I get an query, I add it here to the input "x"

In [3]:
X = np.append(X,[[0.373737, 1, 0], [1, 0, 0.656566], [0, 0, 0.949495], [0.494949, 0, 0.717172]], axis=0)
X

array([[0.17152521, 0.34391687, 0.2487372 ],
       [0.24211446, 0.64407427, 0.27243281],
       [0.53490572, 0.39850092, 0.17338873],
       [0.49258141, 0.61159319, 0.34017639],
       [0.13462167, 0.21991724, 0.45820622],
       [0.34552327, 0.94135983, 0.26936348],
       [0.15183663, 0.43999062, 0.99088187],
       [0.64550284, 0.39714294, 0.91977134],
       [0.74691195, 0.28419631, 0.22629985],
       [0.17047699, 0.6970324 , 0.14916943],
       [0.22054934, 0.29782524, 0.34355534],
       [0.66601366, 0.67198515, 0.2462953 ],
       [0.04680895, 0.23136024, 0.77061759],
       [0.60009728, 0.72513573, 0.06608864],
       [0.96599485, 0.86111969, 0.56682913],
       [0.373737  , 1.        , 0.        ],
       [1.        , 0.        , 0.656566  ],
       [0.        , 0.        , 0.949495  ],
       [0.494949  , 0.        , 0.717172  ]])

Here I'm appending outputs I get from the feedback, so evertime I get the output from the feedback, I add it to the output "y"

In [4]:
y = np.append(y,[-0.169538034, -0.1212037, -0.279407422, -0.19009924], axis=0)

# Fitting in Guassian Distribution

In [5]:
gpr = GaussianProcessRegressor()
gpr.fit(X, y);

# Creating a grid for grid search

In [6]:
x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
x3 = np.linspace(0, 1, 100)

In [7]:
X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        for k in range(len(x3)):
            X_grid.append([x1[i], x2[j], x3[k]])

Implementing the grid search:

In [8]:
X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)

#####################################################################################################
#Below is my previous approach where I used the probability improvement method for the Acquisition Function (you can ignor it)
#ucb = mean + 1.96 * std
#acquisition_function = []
#eta=0.1
#mx = max([-0.169538034, -0.1212037])
#for k in range(len(X_grid)):
 #   acquisition_function.append(1 - NormalDist(mu = mean[k], sigma = std[k]).cdf(mx + eta))
  #  if  mean[k] > mx:
   #     mx = mean[k]
#acquisition_function = np.array(acquisition_function)

# Applying the "Analytic LogEI"

I have found this approach in "Unexpected Improvements to Expected Improvement for Bayesian Optimization" article (expalined in README file)

Now let's define log1mexp which is $$log( 1 - exp(x) )$$

In [9]:
def log1mexp(x):
    return np.log(1-np.exp(x))

$log( 1 - exp(x) )$ is used in the acquistion function later

Recall from README file: 

$$c1 = log(2π) / 2$$
$$c2 =  log(π/2) /2$$
$$erfcx(x) = exp(x^{2})erfc(x)$$
where erfc is the complementary error function.
The "Analytic LogEI" acquistion function is: $$ LogEIy^{∗}(x) = log_h((µ(x) − y^{∗})/σ(x)) + log(σ(x))$$
where $log_h(x)$ is calculated below (based on several conditions mentioned in README file and the article (section 4.1)

eta in the code below is ϵ which is chosen to be 0.1. So Let's apply that below:

In [10]:
z = (mean - max(y)) / std
c1 = np.log(2 * (22/7)) / 2
c2 = np.log((22/7) / 2) / 2
eta=0.1

if (z > -1).any:    # the first condition for log_h
    log_h = np.log(norm.pdf(z) + z * norm.cdf(z) + 1e-10)    # I added "1e-10" to avoid division by zero

elif (-1/np.sqrt(eta) < z).any() and (z <= -1).any():    # the second condition for log_h
    log_h = -z**2/(2-c1) + log1mexp(np.log(sp.erfcx(-z/np.sqrt(2)) * abs(z)) + c2)

elif (z <= -1 / np.sqrt(eta)).any():    # the third condition for log_h
    log_h = -z**2/(2-c1) + 2* np.log(abs(z))

acquisition_function = log_h + np.log(std)
acquisition_function

array([-25.33939508, -25.36642959, -25.3936498 , ...,  -0.76405736,
        -0.75475644,  -0.7463353 ])

# Obtaining the next Query:

In [11]:
idx_max = np.argmax(acquisition_function)
next_query = X_grid[idx_max]
print(next_query.round(6))

[0.767677 1.       0.979798]
